In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
import yaml
from ionpy.util import Config
from ese.scripts.utils import get_option_product
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml calibration_config

data:
  preload: '?' 
  iters_per_epoch: '?' 

dataloader:
  batch_size: 4 
  num_workers: 2
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4
  weight_decay: 0.0 

train:
  epochs: 500 
  eval_freq: 10 
  pretrained_dir: '?'
  pretrained_select_metric: 'val-dice_score'

loss_func: 
  _class: ionpy.loss.PixelCELoss 
  from_logits: True 
  batch_reduction: 'mean' 

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

In [ ]:
%%yaml calibrator_model_cfg  

model:
  _class: '?'
  neighborhood_width: 3

In [ ]:
exp_name = '02_23_24_Shapes_IndSplits_Calibrators' 
group_dict = {
    "dataset": "Shapes",
    "exp_group": exp_name,
    "base_models_group": "02_22_24_Shapes_Tiny",
    "preload": False
}

# exp_name = '02_23_24_Shapes_ShareSplit_Calibrators' 
# group_dict = {
#     "dataset": "Shapes",
#     "exp_group": exp_name,
#     "base_models_group": "02_22_24_Shapes_TinySharedSplit",
#     "preload": False
# }

In [ ]:
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

option_set = get_ese_calibration_configs(
    group_dict=group_dict,
    calibrators=[
        "Vanilla",
        # "TempScaling",
        # "VectorScaling",
        # "DirichletScaling",
        # "LTS",
        # "NectarScaling"
    ],
    base_options=None
)

In [ ]:
# Load the inference cfg from local.
##################################################
cal_cfg_root = code_root / "ese" / "experiment" / "configs" / "calibrate"
##################################################
with open(cal_cfg_root / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_cal_cfg = yaml.safe_load(file)

# Assemble base config
base_cfg = Config(calibration_config).update([calibrator_model_cfg, callbacks_cfg, dataset_cal_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [ ]:
len(cfgs)

## Running Jobs

In [ ]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

In [ ]:
####### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    experiment_class=PostHocExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='3',
)

In [ ]:
# ###### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     experiment_class=PostHocExperiment,
#     track_wandb=True,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )